<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse Supervised
    - Entrainement Supervised
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 14/09/2020 

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
I - Sélection des datas entrantes</p>

In [1]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    import time
    debut_scoring = time.time()
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 20
    if (not mode_supervise_with_lda):
        x_test_list = x_test.tolist()
    for i in range(x_test.shape[0]):

        debut_boucle = time.time()
        #for i in range(x_test.shape[0]):
        #print(x_test)
        text_projection = x_test#[i]#dernier ajout le [i] qui n'y était pas...
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        else:
            text_projection = x_test.values[i]
            text_projection = [text_projection]
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        #voir le temps
        #voir nb tag
        
        debut_prediction = time.time()
        predicted = clf.predict_proba(text_projection)
        fin_prediction = time.time()
        print("temps pour la prediction : {0} s".format(fin_prediction - debut_prediction))
        del debut_prediction, fin_prediction

        '''
        print (predicted)
        print (len(predicted))
        '''
        debut_generation_tag = time.time()
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        fin_generation_tag = time.time()
        print("temps pour la generation : {0} s".format(fin_generation_tag - debut_generation_tag))
        del debut_generation_tag, fin_generation_tag

        #print (str(prediction))
        
        debut_y = time.time()
        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)
        fin_y = time.time()
        print("temps pour lecture y : {0} s".format(fin_y - debut_y))
        del debut_y, fin_y

        debut_ch1 = time.time()
        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        fin_ch1 = time.time()
        print("temps pour check 1 : {0} s".format(fin_ch1 - debut_ch1))
        del debut_ch1, fin_ch1
        
        debut_ch5 = time.time()
        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        fin_ch5 = time.time()
        print("temps pour check 5 : {0} s".format(fin_ch5 - debut_ch5))
        del debut_ch5, fin_ch5
        #str([tag for tag in y_true[i]]if tag ==1)
        fin_boucle = time.time()
        print("temps pour la boucle scorring : {0} s".format(fin_boucle - debut_boucle))
        del fin_boucle, debut_boucle
    fin_scoring = time.time()
    print("temps pour le function scoring : {0} s".format(fin_scoring - debut_scoring))
    del fin_scoring, debut_scoring
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))


In [2]:

import time

In [3]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 
tagText = TagText()

tagText.nombre_post_entree = 50000
tagText.precision = 50000

print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)
debut_lecture = time.time()
data_question = tagText.read_source()
print(data_question.head())

fin_lecture = time.time()
print("temps pour la lecture : {0} s".format(fin_lecture - debut_lecture))
del fin_lecture, debut_lecture

debut_preprocess = time.time()
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
fin_preprocess = time.time()
print("temps pour le pre processing : {0} s".format(fin_preprocess - debut_preprocess))
del fin_preprocess, debut_preprocess

print(data_preprocessed.head())

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Data/
QuestionVsTags.csv
                                                body  \
0  <p>Here is a piece of C++ code that shows some...   
1  <p>I accidentally committed the wrong files to...   
2  <p>I want to delete a branch both locally and ...   
3  <p>What are the differences between <code>git ...   
4  <p>What is the use of the <code>yield</code> k...   

                                               title  \
0  Why is processing a sorted array faster than p...   
1  How do I undo the most recent local commits in...   
2  How do I delete a Git branch locally and remot...   
3  What is the difference between 'git pull' and ...   
4                  What does the "yield" keyword do?   

                                                tags  \
0  <java><c++><performance><optimization><branch-...   
1  <git><version-control><git-commit><undo><pre-c...   
2  <git><version-control><git-branch><git-push><g...   
3        <git><version-control><git-pull><git-fetch>   
4    <python><iterato

In [4]:
def supervised_prepare_tagV2(self, data_preprocessed, data_tag):
    '''
    prepare classifier and class from file for supervised model
    '''
    from sklearn.pipeline import Pipeline
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer

    y_all = [
        item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
        for item in data_tag
    ]

    #print(y_train_tag)
    lb = tagText.MultiLabelBinarizer()
    Y_all = lb.fit_transform(y_all)


    # 80/20 split
    X_train, X_test, y_train, y_test = train_test_split(
        data_preprocessed, y_all, test_size=0.05,train_size=0.95, random_state=0)
    y_train = lb.transform(y_train)
    y_test = lb.transform(y_test)

    classifier = Pipeline([
        ('vectorizer', self.CountVectorizer()),
        ('tfidf', self.TfidfTransformer()),
        (
            'clf',
            self.RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
            )
        )
    ])  
    classifier.fit(X_train, y_train)
    '''
    print (len(X_test))
    print (len(y_test))
    print (y_test[0])
    print (X_test)
    '''
    
    nb1, score1, nb5, score5 = scoring(X_test, y_test, classifier, lb, False, None)
    
    print("score 1 occurence {}".format(score1))
    print("score 5 occurences {}".format(score5))
    
    return classifier, lb.classes_


In [5]:
classifier, tags = supervised_prepare_tagV2(tagText, data_preprocessed, data_question['SelectedTags'])
#del classifier, tags

2500
temps pour la prediction : 1.9018375873565674 s
temps pour la generation : 0.0030298233032226562 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.906825065612793 s
temps pour la prediction : 1.687396764755249 s
temps pour la generation : 0.0019965171813964844 s
temps pour lecture y : 0.0009598731994628906 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.6903531551361084 s
temps pour la prediction : 1.9087870121002197 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.9097850322723389 s
temps pour la prediction : 2.5819520950317383 s
temps pour la generation : 0.0019948482513427734 s
temps pour lecture y : 0.000997781753540039 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 2.584944725036621 s
temps pour la prediction : 1.793104887008

temps pour la prediction : 1.5537571907043457 s
temps pour la generation : 0.001994609832763672 s
temps pour lecture y : 0.0020008087158203125 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.5577526092529297 s
temps pour la prediction : 1.634530782699585 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009980201721191406 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.635528802871704 s
temps pour la prediction : 1.9057996273040771 s
temps pour la generation : 0.0019941329956054688 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.000997781753540039 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.9087915420532227 s
temps pour la prediction : 1.5348091125488281 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.000997781753540039 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.5358068943023682 s
temps pour la prediction : 2.

temps pour la prediction : 1.6544513702392578 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0010042190551757812 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.6564531326293945 s
temps pour la prediction : 1.6036150455474854 s
temps pour la generation : 0.0009975433349609375 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.6056106090545654 s
temps pour la prediction : 1.6195800304412842 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009987354278564453 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.6205787658691406 s
temps pour la prediction : 2.313680648803711 s
temps pour la generation : 0.0019941329956054688 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 2.3156747817993164 s
temps pour la prediction : 1.62656164169311

temps pour la prediction : 1.4769618511199951 s
temps pour la generation : 0.0019960403442382812 s
temps pour lecture y : 0.0009968280792236328 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.479954719543457 s
temps pour la prediction : 1.9207563400268555 s
temps pour la generation : 0.0009989738464355469 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.921755313873291 s
temps pour la prediction : 1.6544826030731201 s
temps pour la generation : 0.0019960403442382812 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.6564786434173584 s
temps pour la prediction : 1.6265804767608643 s
temps pour la generation : 0.0029747486114501953 s
temps pour lecture y : 0.0009970664978027344 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.6305522918701172 s
temps pour la prediction :

temps pour la prediction : 1.0452220439910889 s
temps pour la generation : 0.0009918212890625 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.047215461730957 s
temps pour la prediction : 1.2950150966644287 s
temps pour la generation : 0.0009987354278564453 s
temps pour lecture y : 0.000997781753540039 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2970116138458252 s
temps pour la prediction : 1.2625532150268555 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009968280792236328 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.263550043106079 s
temps pour la prediction : 1.3184008598327637 s
temps pour la generation : 0.0009970664978027344 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3193979263305664 s
temps pour la prediction : 1.1449077129364014 s


...

temps pour la prediction : 1.3222215175628662 s
temps pour la generation : 0.0009968280792236328 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3232183456420898 s
temps pour la prediction : 1.2874367237091064 s
temps pour la generation : 0.0010008811950683594 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.2884376049041748 s
temps pour la prediction : 1.3503315448760986 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3535640239715576 s
temps pour la prediction : 1.369046926498413 s
temps pour la generation : 0.001995563507080078 s
temps pour lecture y : 0.0009968280792236328 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3730359077453613 s
temps pour la prediction : 1.453627586364746 s
temps pour la g

temps pour la prediction : 1.3598666191101074 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3618674278259277 s
temps pour la prediction : 1.3856549263000488 s
temps pour la generation : 0.0010008811950683594 s
temps pour lecture y : 0.0009634494781494141 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3876192569732666 s
temps pour la prediction : 1.3356781005859375 s
temps pour la generation : 0.0009558200836181641 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3376355171203613 s
temps pour la prediction : 1.3428499698638916 s
temps pour la generation : 0.0029985904693603516 s
temps pour lecture y : 0.0019922256469726562 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.3478407859802246 s
temps pour la prediction 

temps pour la prediction : 1.3712730407714844 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009970664978027344 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.375234842300415 s
temps pour la prediction : 1.4967155456542969 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009975433349609375 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.5002293586730957 s
temps pour la prediction : 1.9110829830169678 s
temps pour la generation : 0.0009989738464355469 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.9120819568634033 s
temps pour la prediction : 1.735212802886963 s
temps pour la generation : 0.0029935836791992188 s
temps pour lecture y : 0.0009963512420654297 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.7421596050262451 s
temps pour la prediction : 1.494161367416381

temps pour la prediction : 2.7415807247161865 s
temps pour la generation : 0.004981040954589844 s
temps pour lecture y : 0.0009970664978027344 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 2.747558832168579 s
temps pour la prediction : 3.5419788360595703 s
temps pour la generation : 0.002992391586303711 s
temps pour lecture y : 0.0029888153076171875 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 3.5489556789398193 s
temps pour la prediction : 3.099619150161743 s
temps pour la generation : 0.000997304916381836 s
temps pour lecture y : 0.0009968280792236328 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 3.1016132831573486 s
temps pour la prediction : 5.832483768463135 s
temps pour la generation : 0.0019953250885009766 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 5.834479093551636 s
temps pour la

temps pour la prediction : 1.0831656455993652 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0009946823120117188 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.084160327911377 s
temps pour la prediction : 1.0862782001495361 s
temps pour la generation : 0.000997781753540039 s
temps pour lecture y : 0.002425670623779297 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 1.0897016525268555 s
temps pour la prediction : 0.9953622817993164 s
temps pour la generation : 0.0 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 0.9963650703430176 s
temps pour la prediction : 0.9793658256530762 s
temps pour la generation : 0.0009686946868896484 s
temps pour lecture y : 0.0 s
temps pour check 1 : 0.0 s
temps pour check 5 : 0.0 s
temps pour la boucle scorring : 0.9803345203399658 s
temps pour la prediction : 0.9483659267425537 s
temps pour la g

In [6]:
classifier, tags = tagText.supervised_prepare_tagV2(data_preprocessed, data_question['SelectedTags'])
#del classifier, tags



temps pour la boucle scorring : 1.0721240043640137 s
temps pour la boucle scorring : 1.115006446838379 s
temps pour la boucle scorring : 0.9554653167724609 s
temps pour la boucle scorring : 0.9275157451629639 s
temps pour la boucle scorring : 1.0371794700622559 s
temps pour la boucle scorring : 0.8985860347747803 s
temps pour la boucle scorring : 0.9065980911254883 s
temps pour la boucle scorring : 0.8846282958984375 s
temps pour la boucle scorring : 0.8766486644744873 s
temps pour la boucle scorring : 0.8855855464935303 s
temps pour la boucle scorring : 0.8746519088745117 s
temps pour la boucle scorring : 0.8800687789916992 s
temps pour la boucle scorring : 0.8866145610809326 s
temps pour la boucle scorring : 0.8816046714782715 s
temps pour la boucle scorring : 0.8616831302642822 s
temps pour la boucle scorring : 1.0312330722808838 s
temps pour la boucle scorring : 0.8921101093292236 s
temps pour la boucle scorring : 1.0236704349517822 s
temps pour la boucle scorring : 1.0561628341674

temps pour la boucle scorring : 0.9015800952911377 s
temps pour la boucle scorring : 0.8786697387695312 s
temps pour la boucle scorring : 0.8776447772979736 s
temps pour la boucle scorring : 0.8915984630584717 s
temps pour la boucle scorring : 0.9404826164245605 s
temps pour la boucle scorring : 1.2366411685943604 s
temps pour la boucle scorring : 1.0073299407958984 s
temps pour la boucle scorring : 0.8936002254486084 s
temps pour la boucle scorring : 0.9952964782714844 s
temps pour la boucle scorring : 1.0043039321899414 s
temps pour la boucle scorring : 0.9200398921966553 s
temps pour la boucle scorring : 0.9693992137908936 s
temps pour la boucle scorring : 1.0183119773864746 s
temps pour la boucle scorring : 1.088078498840332 s
temps pour la boucle scorring : 0.913546085357666 s
temps pour la boucle scorring : 0.8776757717132568 s
temps pour la boucle scorring : 0.9239959716796875 s
temps pour la boucle scorring : 0.9414725303649902 s
temps pour la boucle scorring : 0.92753720283508

temps pour la boucle scorring : 0.9265110492706299 s
temps pour la boucle scorring : 0.8666729927062988 s
temps pour la boucle scorring : 0.8906490802764893 s
temps pour la boucle scorring : 0.9241015911102295 s
temps pour la boucle scorring : 0.9504480361938477 s
temps pour la boucle scorring : 0.8746519088745117 s
temps pour la boucle scorring : 0.9464609622955322 s
temps pour la boucle scorring : 1.0332262516021729 s
temps pour la boucle scorring : 0.9405050277709961 s
temps pour la boucle scorring : 0.9335012435913086 s
temps pour la boucle scorring : 0.9404850006103516 s
temps pour la boucle scorring : 0.8826627731323242 s
temps pour la boucle scorring : 0.9384503364562988 s
temps pour la boucle scorring : 0.932544469833374 s
temps pour la boucle scorring : 0.980402946472168 s
temps pour la boucle scorring : 0.8836231231689453 s
temps pour la boucle scorring : 0.9384505748748779 s
temps pour la boucle scorring : 0.9085590839385986 s
temps pour la boucle scorring : 0.88366436958312

temps pour la boucle scorring : 0.8812131881713867 s
temps pour la boucle scorring : 0.9334914684295654 s
temps pour la boucle scorring : 0.9225547313690186 s
temps pour la boucle scorring : 0.9703941345214844 s
temps pour la boucle scorring : 0.9913067817687988 s
temps pour la boucle scorring : 0.9464588165283203 s
temps pour la boucle scorring : 0.9534718990325928 s
temps pour la boucle scorring : 0.963381290435791 s
temps pour la boucle scorring : 0.8636808395385742 s
temps pour la boucle scorring : 1.0521774291992188 s
temps pour la boucle scorring : 0.9509549140930176 s
temps pour la boucle scorring : 0.8816308975219727 s
temps pour la boucle scorring : 0.9544379711151123 s
temps pour la boucle scorring : 0.8877513408660889 s
temps pour la boucle scorring : 0.9893438816070557 s
temps pour la boucle scorring : 0.925513744354248 s
temps pour la boucle scorring : 0.8791465759277344 s
temps pour la boucle scorring : 0.884617805480957 s
temps pour la boucle scorring : 0.889643192291259

temps pour la boucle scorring : 0.9035725593566895 s
temps pour la boucle scorring : 0.8906404972076416 s
temps pour la boucle scorring : 0.8912174701690674 s
temps pour la boucle scorring : 0.8926043510437012 s
temps pour la boucle scorring : 0.9275119304656982 s
temps pour la boucle scorring : 1.0517163276672363 s
temps pour la boucle scorring : 0.8954665660858154 s
temps pour la boucle scorring : 0.8835885524749756 s
temps pour la boucle scorring : 1.0222904682159424 s
temps pour la boucle scorring : 0.8876152038574219 s
temps pour la boucle scorring : 0.8676388263702393 s
temps pour la boucle scorring : 0.9175732135772705 s
temps pour la boucle scorring : 0.9464242458343506 s
temps pour la boucle scorring : 0.9255950450897217 s
temps pour la boucle scorring : 0.8886203765869141 s
temps pour la boucle scorring : 0.9065921306610107 s
temps pour la boucle scorring : 0.8746201992034912 s
temps pour la boucle scorring : 0.8751585483551025 s
temps pour la boucle scorring : 0.880667448043

...

temps pour la boucle scorring : 0.9355287551879883 s
temps pour la boucle scorring : 0.8801124095916748 s
temps pour la boucle scorring : 0.876704216003418 s
temps pour la boucle scorring : 0.8965973854064941 s
temps pour la boucle scorring : 0.8956015110015869 s
temps pour la boucle scorring : 0.9100768566131592 s
temps pour la boucle scorring : 0.904578685760498 s
temps pour la boucle scorring : 0.9295403957366943 s
temps pour la boucle scorring : 0.8899178504943848 s
temps pour la boucle scorring : 0.8985650539398193 s
temps pour la boucle scorring : 0.8962116241455078 s
temps pour la boucle scorring : 1.0122814178466797 s
temps pour la boucle scorring : 0.8926105499267578 s
temps pour la boucle scorring : 0.8944144248962402 s
temps pour la boucle scorring : 0.8846626281738281 s
temps pour la boucle scorring : 0.8766193389892578 s
temps pour la boucle scorring : 0.895632266998291 s
temps pour la boucle scorring : 1.0511541366577148 s
temps pour la boucle scorring : 0.914587497711181

temps pour la boucle scorring : 0.9504237174987793 s
temps pour la boucle scorring : 0.8976309299468994 s
temps pour la boucle scorring : 0.8646600246429443 s
temps pour la boucle scorring : 1.002316951751709 s
temps pour la boucle scorring : 0.9205663204193115 s
temps pour la boucle scorring : 0.8866219520568848 s
temps pour la boucle scorring : 0.8726356029510498 s
temps pour la boucle scorring : 0.9425067901611328 s
temps pour la boucle scorring : 0.886624813079834 s
temps pour la boucle scorring : 0.9693717956542969 s
temps pour la boucle scorring : 0.8587276935577393 s
temps pour la boucle scorring : 1.0741243362426758 s
temps pour la boucle scorring : 0.8866231441497803 s
temps pour la boucle scorring : 0.943505048751831 s
temps pour la boucle scorring : 0.8786139488220215 s
temps pour la boucle scorring : 0.9009687900543213 s
temps pour la boucle scorring : 1.03519606590271 s
temps pour la boucle scorring : 0.9375200271606445 s
temps pour la boucle scorring : 0.89955735206604 s


temps pour la boucle scorring : 0.896599292755127 s
temps pour la boucle scorring : 0.8925755023956299 s
temps pour la boucle scorring : 0.9012696743011475 s
temps pour la boucle scorring : 1.0202372074127197 s
temps pour la boucle scorring : 0.8736579418182373 s
temps pour la boucle scorring : 0.8856594562530518 s
temps pour la boucle scorring : 0.9435014724731445 s
temps pour la boucle scorring : 0.9150738716125488 s
temps pour la boucle scorring : 0.9035751819610596 s
temps pour la boucle scorring : 1.020235300064087 s
temps pour la boucle scorring : 1.0322365760803223 s
temps pour la boucle scorring : 1.074122667312622 s
temps pour la boucle scorring : 0.9091012477874756 s
temps pour la boucle scorring : 0.9044418334960938 s
temps pour la boucle scorring : 0.8896193504333496 s
temps pour la boucle scorring : 0.8916075229644775 s
temps pour la boucle scorring : 0.8905823230743408 s
temps pour la boucle scorring : 0.9814074039459229 s
temps pour la boucle scorring : 0.997342348098754

temps pour la boucle scorring : 0.8836209774017334 s
temps pour la boucle scorring : 0.9863595962524414 s
temps pour la boucle scorring : 0.9953334331512451 s
temps pour la boucle scorring : 0.9634571075439453 s
temps pour la boucle scorring : 0.9135446548461914 s
temps pour la boucle scorring : 0.9519810676574707 s
temps pour la boucle scorring : 0.9594283103942871 s
temps pour la boucle scorring : 1.0461995601654053 s
temps pour la boucle scorring : 1.0198092460632324 s
temps pour la boucle scorring : 0.8984599113464355 s
temps pour la boucle scorring : 0.9803750514984131 s
temps pour la boucle scorring : 1.0671417713165283 s
temps pour la boucle scorring : 0.9873745441436768 s
temps pour la boucle scorring : 0.9653825759887695 s
temps pour la boucle scorring : 1.0021343231201172 s
temps pour la boucle scorring : 1.0471620559692383 s
temps pour la boucle scorring : 0.9265165328979492 s
temps pour la boucle scorring : 1.0242576599121094 s
temps pour la boucle scorring : 1.019777297973

temps pour la boucle scorring : 0.9554276466369629 s
temps pour la boucle scorring : 0.9385159015655518 s
temps pour la boucle scorring : 0.8851795196533203 s
temps pour la boucle scorring : 0.901592493057251 s
temps pour la boucle scorring : 0.8965916633605957 s
temps pour la boucle scorring : 0.9184954166412354 s
temps pour la boucle scorring : 1.0421717166900635 s
temps pour la boucle scorring : 0.9135839939117432 s
temps pour la boucle scorring : 0.8782191276550293 s
temps pour la boucle scorring : 0.8894503116607666 s
temps pour la boucle scorring : 0.8845982551574707 s
temps pour le function scoring : 2332.1412012577057 s
score 1 occurence 2.96
score 5 occurences 0.08


In [7]:
data_preprocessed[0]

'piece c code shows peculiar behavior strange reason sorting data miraculously makes code almost six times faster include algorithm include ctime include iostream int main   generate data const unsigned arraysize  32768 int dataarraysize unsigned c  0 c  arraysize c datac  stdrand  256   this next loop runs faster stdsortdata data  arraysize  test clockt start  clock long long sum  0 unsigned  0  100000 i   primary loop unsigned c  0 c  arraysize c  datac  128 sum  datac   double elapsedtime  staticcastdoubleclock  start  clockspersec stdcout  elapsedtime  stdendl stdcout  sum    sum  stdendl  without stdsortdata data  arraysize code runs 1154 seconds sorted data code runs 193 seconds initially thought might language compiler anomaly tried java import javautilarrays import javautilrandom public class main  public static void mainstring args   generate data int arraysize  32768 int data  new intarraysize random rnd  new random0 int c  0 c  arraysize c datac  rndnextint  256   this next 

In [8]:
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

y_all = [
    item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
    for item in data_question['SelectedTags']
]

#print(y_train_tag)
lb = tagText.MultiLabelBinarizer()
Y_all = lb.fit_transform(y_all)


# 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    data_preprocessed, y_all, test_size=0.05,train_size=0.95, random_state=0)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

In [9]:
def supervised_predict(model, text, classifier, classes, no_top_words=5):
    '''
    predict tag form text in function of supervised model
    '''
    print(text)
    print([text])
    predicted = classifier.predict_proba([text])
    tempTag = [(1-item[0][0]) for item in predicted]
    list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]

    list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])

    list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]

    return str([classes[id] for id in list_id_sorted_suggested])
    
supervised_predict(tagText, 'c c# .Net', classifier, tags )

c c# .Net
['c c# .Net']


"['google-chrome', 'debugging', 'ajax', 'python-3.x', 'dataframe']"

In [10]:

message = 'git est un outils de code source. C\'est bien'
print(supervised_predict(tagText, message, classifier, tags))

git est un outils de code source. C'est bien
["git est un outils de code source. C'est bien"]
['google-chrome', 'debugging', 'ajax', 'python-3.x', 'dataframe']


In [11]:

message = 'git est un outils de code source. C\'est bien'
print(message)



print(tagText.supervised_predict(message, classifier, tags))

git est un outils de code source. C'est bien
['google-chrome', 'debugging', 'ajax', 'python-3.x', 'dataframe']


In [12]:
print(tagText.supervised_predict(data_preprocessed[0], classifier, tags ))

['exception', 'python-3.x', 'debugging', 'ajax', 'dataframe']


In [13]:
print (tags)

['.htaccess' '.net' '.net-core' '3d' 'actionscript-3' 'active-directory'
 'activerecord' 'ajax' 'algorithm' 'amazon-ec2' 'amazon-s3'
 'amazon-web-services' 'anaconda' 'android' 'android-activity'
 'android-asynctask' 'android-edittext' 'android-emulator'
 'android-fragments' 'android-gradle-plugin' 'android-intent'
 'android-layout' 'android-listview' 'android-ndk' 'android-recyclerview'
 'android-studio' 'android-viewpager' 'angular' 'angular-material'
 'angular-ui-router' 'angularjs' 'angularjs-directive' 'animation'
 'annotations' 'ansible' 'ant' 'apache' 'apache-flex' 'apache-kafka'
 'apache-spark' 'apache-spark-sql' 'api' 'architecture' 'arduino' 'arm'
 'arraylist' 'arrays' 'asp-classic' 'asp.net' 'asp.net-core'
 'asp.net-core-mvc' 'asp.net-mvc' 'asp.net-mvc-3' 'asp.net-mvc-4'
 'asp.net-mvc-5' 'asp.net-web-api' 'assembly' 'async-await' 'asynchronous'
 'attributes' 'audio' 'authentication' 'autocomplete' 'autolayout'
 'automated-tests' 'automation' 'awk' 'aws-lambda' 'axios' 'azure

In [14]:
tagText.supervised_predict('c c# .Net', classifier, tags )

"['google-chrome', 'debugging', 'ajax', 'python-3.x', 'dataframe']"

In [15]:
tagText.supervised_predict('how to convert int in flaot in c c# .Net', classifier, tags, 120000 )

"['google-chrome', 'debugging', 'ajax', 'python-3.x', 'dataframe', 'scala', 'go', 'oop', 'intellij-idea', 'linq', 'command-line', 'syntax', 'cocoa-touch', 'wpf', 'spring', 'xml', 'twitter-bootstrap', 'android-studio', 'file', 'unit-testing', 'c++11', 'http', 'tsql', 'dictionary', 'docker', 'algorithm', 'date', 'unix', 'typescript', 'multithreading', 'datetime', 'vim', 'postgresql', 'github', 'pandas', 'reactjs', 'angular', 'database', 'macos', 'eclipse', 'performance', 'windows', 'visual-studio', 'asp.net-mvc', 'list', 'xcode', 'asp.net', 'django', 'regex', 'r', 'shell', 'angularjs', 'iphone', 'json', 'ruby-on-rails', 'swift', 'sql-server', 'ruby', 'linux', 'objective-c', 'arrays', 'bash', 'c', 'node.js', 'mysql', 'string', 'sql', 'php', 'ios', '.net', 'css', 'jquery', 'git', 'c++', 'html', 'android', 'c#', 'java', 'python', 'javascript']"

In [16]:
message = 'I try to put an int into a float and I cannot. Please help me to convert int into float in c'
print(message)



print(tagText.supervised_predict(message, classifier, tags ))

I try to put an int into a float and I cannot. Please help me to convert int into float in c
['google-chrome', 'debugging', 'ajax', 'python-3.x', 'dataframe']
